# DBMS Application

Here is a Python DBMS application abiding by the restrictions set in the COSC 5510 Project 3 Instructions.

# README:
Just run the notebook from top to bottom and all the necessary packages should be installed!

Make sure that the relevant csv files are located in the same folder as the notebook, or if using Google Colab then dragged into the file storage area. (relation1, relation2, relation3, relation4, vistors, rides)

There are example queries at the end of the document to run. To exit the program run "exit;"

# Package setup
[all packages have been approved by TAs in writing]

In [ ]:
# Install necessary packages for this project
# input and output formatting
import csv
from tabulate import tabulate
from collections import defaultdict

# indexing
!pip install btrees
from BTrees.OOBTree import OOBTree

# execution time
import time

# Packages for parsing and command line interface
import os
import sys
import re
import time
import sqlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.4/231.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 5.0 MB/s eta 0:00:00


# Constant Datapaths

In [ ]:
DATA_FILE_PATH1 = 'relation1.csv'
DATA_FILE_PATH2 = 'relation2.csv'
DATA_FILE_PATH3 = 'relation3.csv'
DATA_FILE_PATH4 = 'relation4.csv'
DATA_FILE_PATH5 = 'visitors.csv'
DATA_FILE_PATH6 = 'rides.csv'

# Global Vars

In [ ]:
db = {} # DON'T REMOVE - HAS TABLE NAMES AND COLUMNS FOR DATABASE
keys = {}
file_paths = {
    'data_1': 'relation1.csv',
    'data_2': 'relation2.csv',
    'data_3': 'relation3.csv',
    'data_4': 'relation4.csv',
    'visitors': 'visitors.csv',
    'rides': 'rides.csv'
    }

# Loading in datasets from CSV Files
Can be updated to bulk upload.

In [ ]:
def load_data(filename, db_name):
    with open(filename, mode='r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Extracting column names
        data = [row for row in reader]

    return {db_name: {'headers': headers, 'data': data}}

In [ ]:
'''def given_sample_upload():

    # Loading the data from CSVs to our program
    db_1 = load_data(DATA_FILE_PATH1, "data_1")
    db_2 = load_data(DATA_FILE_PATH2, "data_2")
    db_3 = load_data(DATA_FILE_PATH2, "data_3")
    db_4 = load_data(DATA_FILE_PATH2, "data_4")
    file_paths = {
    'data_1': 'relation1.csv',
    'data_2': 'relation2.csv',
    'data_3': 'relation3.csv',
    'data_4': 'relation4.csv',
    }

    # Combining databases into one dictionary for easier access
    databases = {**db_1, **db_2, **db_3, **db_4}

    # Adding tables to db - DON'T REMOVE THIS
    db['data_1'] = ['col1', 'col2']
    db['data_2'] = ['col1', 'col2']
    db['data_3'] = ['col1', 'col2']
    db['data_4'] = ['col1', 'col2']

    # Displaying the data with headers for each dataset
    for db_name, contents in databases.items():
        print(f"\n{db_name}:")
        # Slicing the data to display only the first 10 rows
        first_10_rows = contents['data'][:10]
        print(tabulate(first_10_rows, headers=contents['headers'], tablefmt="grid"))

    return databases, file_paths
'''

def create_data(rows, pattern=1):
  # Pattern - 1 : [[1, 1], [2, 2], ... ]
  # Pattern - 2 : [[1, 1], [2, 1], ... ]
  return [[i, i if pattern == 1 else 1] for i in range(1, rows + 1)]

def given_sample_upload():
  # Creating tables with headers and data
  d1 = {'headers': ['col1', 'col2'], 'data': create_data(1000)}
  d2 = {'headers': ['col1', 'col2'], 'data': create_data(1000, pattern=2)}
  d3 = {'headers': ['col1', 'col2'], 'data': create_data(10000, pattern=1)}
  d4 = {'headers': ['col1', 'col2'], 'data': create_data(10000, pattern=2)}
  # d5 = load_data(DATA_FILE_PATH5, "visitors")
  # d6 = load_data(DATA_FILE_PATH6, "rides")
  keys['data_1'] = {'pk': 'col1', 'fk': ''}
  keys['data_2'] = {'pk': 'col1', 'fk': ''}
  keys['data_3'] = {'pk': 'col1', 'fk': ''}
  keys['data_4'] = {'pk': 'col1', 'fk': ''}
  keys['visitors'] = {'pk': 'visitor_id', 'fk': ''}
  keys['rides'] = {'pk': 'ride_id', 'fk': 'visitor_id'}
  # Combining databases into one dictionary for easier access
  databases = {'data_1': d1, 'data_2': d2, 'data_3': d3, 'data_4': d4}
  # Adding tables to db - DON'T REMOVE THIS
  db['data_1'] = ['col1', 'col2']
  db['data_2'] = ['col1', 'col2']
  db['data_3'] = ['col1', 'col2']
  db['data_4'] = ['col1', 'col2']
  # db['visitors'] = ['visitor_id', 'name', 'age', 'favourite_character']
  # db['rides'] = ['ride_id', 'visitor_id', 'ride_name', 'times_ridden']
  # Displaying the data with headers for each dataset
  '''for db_name, contents in databases.items():
    print(f"\n{db_name}:")
    # Slicing the data to display only the first 10 rows
    first_10_rows = contents['data'][:10]
    print(tabulate(first_10_rows, headers=contents['headers'], tablefmt="grid"))'''
  return databases, file_paths

In [ ]:
databases, file_paths = given_sample_upload()
# output will be first 10 rows of each database

# Parsing Setup Code

Functions all related to parser

In [ ]:
# A function to get token positions in the parsed query
def positions(command):
  pos = {}
  cmd = str(command[0]).split()
  keys = {
        'select', 'distinct', 'as', 'count', 'max', 'min', 'avg', 'sum', 'from',
        'right', 'left', 'join', 'on', 'where', 'and', 'or', 'not', 'in', 'between',
        'like', 'group', 'having', 'order', 'limit', 'create', 'table', 'if', 'exists',
        'insert', 'into', 'values', 'update', 'set', 'delete', 'drop', 'column', 'table', 'alter',
        'add'
        }
  pos = {k: i for i, k in enumerate(cmd) if k in keys or k == ';'}
  # print(pos)
  return pos

# A function to preprocess and clean the parsed query
def clean(values):
  value = []
  chars_to_remove = "();',"
  for v in values:
    val = ''.join(char for char in v if char not in chars_to_remove).strip()
    value.append(val)
  return value

# A function to convert data types of parsed query
def format(fields):
    result = {}
    data_type = {'int': 'int', 'text': 'string'}
    for i in range(0, len(fields), 2):
        key = fields[i]
        original_value = fields[i + 1]
        normalized_value = re.sub(r'\d+|\(.*\)', '', original_value).strip()
        if 'char' in normalized_value or 'varchar' in normalized_value:
            value = 'string'
        else:
            value = data_type.get(normalized_value, original_value)
        result[key] = value
    return result

# A function to convert lists to dictionaries for manipulating data
def format_values(columns, values):
   c = len(columns)
   v = len(values)
   result = {}
   if c == v:
    for i in range(0,c):
      result[columns[i]] = values[i]
    return result
   else:
    print('Missing fields')
    return None

# A function to check duplicate values
def duplicates(columns):
  cols = []
  d = False
  for i in columns:
    if i not in cols:
      cols.append(i)
    else:
      d = True
  return d

# Indexing using BTrees
Currently indexing entire dataset. Can change to just PKs.
Includes search, update and delete.

***need to add updating the file paths when these changes occur***

In [ ]:
class DynamicDB:
    def __init__(self, databases):
        self.datasets = databases
        self.indices = {}

    def load_indices(self):
        """Load indices for each dataset based on a predefined primary key for each dataset."""
        for db_name, contents in self.datasets.items():
            headers = contents['headers']
            pk = keys[db_name]['pk']  # Assuming 'keys' dict defines primary keys
            self.create_indices(db_name, contents['data'], headers, [pk])

    def create_indices2(self, dataset_id, data, headers, index_fields):
        if dataset_id not in self.indices:
            self.indices[dataset_id] = {}
        for field in index_fields:
            if field not in self.indices[dataset_id]:
                self.indices[dataset_id][field] = OOBTree()
            for i, row in enumerate(data):
                key = row[headers.index(field)]
                if key not in self.indices[dataset_id][field]:
                    self.indices[dataset_id][field][key] = []
                self.indices[dataset_id][field][key].append(i)  # Store indices of rows

    def create_indices(self, dataset_id, data, headers, index_fields):
      if dataset_id not in self.indices:
        self.indices[dataset_id] = {}
      for field in index_fields:
        if field not in self.indices[dataset_id]:
            self.indices[dataset_id][field] = OOBTree()
        for i, row in enumerate(data):
            key = str(row[headers.index(field)])  # Cast key to string to avoid type issues
            if key not in self.indices[dataset_id][field]:
                self.indices[dataset_id][field][key] = []
            self.indices[dataset_id][field][key].append(i)


    def check_index(self, dataset_id, field):
        """Ensure the field is indexed, index on-the-fly if it's not."""
        if dataset_id in self.datasets and dataset_id not in self.indices:
            headers = self.datasets[dataset_id]['headers']
            self.create_indices(dataset_id, self.datasets[dataset_id]['data'], headers, [field])
        elif field not in self.indices.get(dataset_id, {}):
            print(f"Indexing field '{field}' because it was needed.")
            self.create_indices(dataset_id, self.datasets[dataset_id]['data'], self.datasets[dataset_id]['headers'], [field])

    def query_by_index2(self, dataset_id, field, value):
        """Query rows based on index if the column is indexed."""
        self.check_index(dataset_id, field)  # Ensure index exists before querying
        if field in self.indices.get(dataset_id, {}):
            indices = self.indices[dataset_id][field].get(value, [])
            return [self.datasets[dataset_id]['data'][i] for i in indices]
        else:
            print(f"No index found for {field} with value {value}.")
            return []

    def query_by_index(self, dataset_id, field, value):
      """Query rows based on index if the column is indexed."""
      self.check_index(dataset_id, field)  # Ensure index exists before querying
      if field in self.indices.get(dataset_id, {}):
        indices = self.indices[dataset_id][field].get(value, [])
        # Flatten the list if it's a list of lists (which should not be the case ideally)
        # This is just a safeguard; your actual implementation should not require this.
        if indices and isinstance(indices[0], list):
            indices = [item for sublist in indices for item in sublist]
        return indices
      else:
        print(f"No index found for {field} with value {value}.")
        return []


# Query Functions

Print a specfic table

In [ ]:
def print_table(db_dict, table_name):
    if table_name in db_dict:
        table_data = db_dict[table_name]
        print(f"\n{table_name}:")
        print(tabulate(table_data['data'], headers=table_data['headers'], tablefmt="grid"))
    else:
        print(f"Table '{table_name}' not found.")

# eg
# print_table(databases, "Data_1")

# Update function
[Not indexed]

In [ ]:
def update_database(databases, db_name, row_index, column_index, new_value):

    if db_name in databases:
        database = databases[db_name]
        data = database['data']

        if row_index < len(data) and column_index < len(data[row_index]):
            data[row_index][column_index] = new_value
            print(f"Updated '{db_name}' at row {row_index}, column {column_index} with new value: {new_value}")
            return True
        else:
            print("Error: Row index or column index out of range.")
            return False
    else:
        print(f"Error: Database named '{db_name}' not found.")
        return False

"""
# example usage
# Update the database
db_name = 'Data_1'
row_index = 1  # Second row
column_index = 1  # Second column (zero-indexed)
new_value = '9'  # New value to set

# Assuming all_databases has been defined and loaded as per our earlier discussions
# Since column names aren't specified, we use indices; modify the update function accordingly if needed
success = update_database(databases, db_name, row_index, column_index, new_value)
if success:
    print("Update successful.")
else:
    print("Update failed.")
print_table(databases, "Data_1") """

'\n# example usage\n# Update the database\ndb_name = \'Data_1\'\nrow_index = 1  # Second row\ncolumn_index = 1  # Second column (zero-indexed)\nnew_value = \'9\'  # New value to set\n\n# Assuming all_databases has been defined and loaded as per our earlier discussions\n# Since column names aren\'t specified, we use indices; modify the update function accordingly if needed\nsuccess = update_database(databases, db_name, row_index, column_index, new_value)\nif success:\n    print("Update successful.")\nelse:\n    print("Update failed.")\nprint_table(databases, "Data_1") '

# Update Parser Code

In [ ]:
dynamic_db = DynamicDB(databases);
dynamic_db.load_indices()  # Load or reload indices to ensure all are set

def update_data_archive(database, dynamic_db, table_name, setcol, setval, col, val):
    # Check if the table exists in the database
    if table_name not in database:
        print(f"Table '{table_name}' not found.")
        return

    # Perform a query using indices
    dynamic_db.check_index(table_name, col)
    matched_indices = dynamic_db.query_by_index(table_name, col, val)
    if not matched_indices:
        print(f"No records found where {col} = {val}.")
        return

    # Retrieve table data
    table = database[table_name]
    headers = table['headers']
    data = table['data']

    # Find the indices of the columns
    setcol_index = headers.index(setcol)
    col_index = headers.index(col)

    # Update the rows that match the condition
    updated_rows = 0
    for index in matched_indices:
        row = data[index]
        if str(row[col_index]) == val:  # Ensure value matches before updating
            old_value = row[setcol_index]  # Capture old value for index update if needed
            row[setcol_index] = setval
            updated_rows += 1
            # Update index if the modified column is indexed
            if setcol in dynamic_db.indices[table_name]:
                dynamic_db.indices[table_name][setcol].pop(old_value, None)  # Remove old index entry
                if setval not in dynamic_db.indices[table_name][setcol]:
                    dynamic_db.indices[table_name][setcol][setval] = []
                dynamic_db.indices[table_name][setcol][setval].append(index)

    # Feedback to the user about the operation
    print(f"Updated {updated_rows} row(s) where {col} = {val} setting {setcol} to {setval}.")

    # Optionally, debug print to confirm data change
    print(f"First few records post-update: {data[:4]}")  # Print first few records to confirm


def update_data(database, dynamic_db, table_name, setcol, setval, col, val):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")
        return

    dynamic_db.check_index(table_name, col)
    matched_indices = dynamic_db.query_by_index(table_name, col, val)
    if not matched_indices:
        print(f"No records found where {col} = {val}.")
        return

    table = database[table_name]
    headers = table['headers']
    data = table['data']

    setcol_index = headers.index(setcol)
    col_index = headers.index(col)

    updated_rows = 0
    for index in matched_indices:
        row = data[index]
        if str(row[col_index]) == val:  # Check matching condition
            # Convert types for consistency in indices
            old_value = str(row[setcol_index])
            new_value = str(setval)

            row[setcol_index] = new_value
            updated_rows += 1

            # Update index if the modified column is indexed
            if setcol in dynamic_db.indices[table_name]:
                # Remove old value entry from index
                if old_value in dynamic_db.indices[table_name][setcol]:
                    dynamic_db.indices[table_name][setcol][old_value].remove(index)
                    if not dynamic_db.indices[table_name][setcol][old_value]:  # Clean up if empty
                        del dynamic_db.indices[table_name][setcol][old_value]

                # Add new value entry to index
                if new_value not in dynamic_db.indices[table_name][setcol]:
                    dynamic_db.indices[table_name][setcol][new_value] = []
                dynamic_db.indices[table_name][setcol][new_value].append(index)

    print(f"Updated {updated_rows} row(s) where {col} = {val} setting {setcol} to {setval}.")

In [ ]:
# Update table query parse implementation
def update(command, pos):
  tables = [] # Stores table names
  setvalue = [] # Stores values that have to be updated
  values = [] # Stores condition of the values that are to be updated
  cmd = str(command).split()
  if pos['update'] == 0 and pos['set'] == 2 and pos['where']:
    tables = cmd[1]
    setvalue = cmd[3:pos['where']]
    setvalue = clean(setvalue)
    values = cmd[pos['where']+1:len(cmd)]
    values = clean(values)
    c = setvalue.index('=')
    setcol, setval = setvalue[0:c], setvalue[c+1:len(setvalue)]
    setcol = setcol[0]
    setval = setval[0]
    c = values.index('=')
    col, val = values[0:c], values[c+1:len(values)]
    col = col[0]
    val = val[0]
    if tables not in db.keys(): # Checking if the table exists
      print('Table not found')
    elif setcol not in db[tables]: # Checking if the columns exists
      print('Column not found')
    elif col not in db[tables]: # Checking if the columns exists
      print('Column not found')
    else: # Updating data
      # *************** Update table function implementation - use tables(list), setvalue(dict) and values(dict) as parameters
      update_data(databases, dynamic_db, tables, setcol, setval, col, val)
      pass
  else:
    print('Invalid Query')
  return None

# Delete Record Function

In [ ]:
def delete_data(database, table_name, column_name, value):
    if table_name not in database:
        print("Table not found")
        return

    # Accessing the table
    table = database[table_name]
    headers = table['headers']

    if column_name not in headers:
        print("Column not found")
        return

    # Finding the index of the column to check
    column_index = headers.index(column_name)

    # Filtering out rows that match the deletion criterion
    original_row_count = len(table['data'])
    table['data'] = [row for row in table['data'] if str(row[column_index]) != str(value)]

    if len(table['data']) < original_row_count:
        print(f"Deleted {original_row_count - len(table['data'])} records where {column_name} is {value}")
    else:
        print("No records found to delete")

# Delete parser code

In [ ]:
def delete(command, pos):
  tables = [] # Stores table names
  delvalue = [] # Stores values to be deleted
  cmd = str(command).split()
  if pos['delete'] == 0 and pos['from'] == 1 and pos['where'] == 3:
    tables = cmd[2]
    delvalue = cmd[4:len(cmd)]
    delvalue = clean(delvalue)
    c = delvalue.index('=')
    col, val = delvalue[0:c], delvalue[c+1:len(delvalue)]
    col = col[0]
    val = val[0]
    if tables not in db.keys(): # Checking if the table exists
      print('Table not found')
    elif col not in db[tables]: # Checking if the columns exists
      print('Column not found')
    else: # Deleting data
      # *************** Delete table function implementation - use tables(list), col and val as parameters
      delete_data(databases, tables, col, val)
  else:
    print('Invalid Query')
  return None

# Dropping a Table

In [ ]:
def drop_table(db, table_name):
    if table_name in db:
        del db[table_name]
        print("Table '{table_name}' has been dropped successfully.")
    else:
        return f"Error: Table '{table_name}' does not exist in the database."

    # List current tables in the database
    if databases:
        current_tables = ', '.join(db.keys())
        # print(f"Current tables in database: {current_tables}")
    else:
        print("No tables left in the database.")


# Dropping a table parser function

In [ ]:
def drop(command, databases):
    cmd = command.split()
    if cmd[0].lower() == 'drop' and cmd[1].lower() == 'table':
        table_name = cmd[2]
        table_name = table_name.replace(';', '')  # Remove any trailing semicolons
        # print(f"Attempting to drop table: {table_name}")
        print(databases)
        if table_name not in db.keys():
          print('Table not found')
        else:
          del db[table_name]
          drop_table(databases, table_name)
    else:
        print('Invalid Query')


Select * function

In [ ]:
def display_all_tables(databases):
    for db_name, contents in databases.items():
        print(f"\n{db_name}:")
        print(tabulate(contents['data'], headers=contents['headers'], tablefmt="grid"))


Select all parser implementation

In [ ]:
def show_tables(command, databases):
    print(str(command).strip().lower())
    if str(command).strip().lower() == 'show tables;':
        display_all_tables(db)
    else:
        print('Invalid Query')

Order by function

In [ ]:
def order_by(database, table_name, column_name, ascending=True):
    if table_name not in database:
        raise ValueError(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
        print(f"Column '{column_name}' not found in the dataset.")

    column_index = headers.index(column_name)
    # Sort the data based on the specified column
    sorted_data = sorted(data, key=lambda x: x[column_index] if x[column_index] is not None else float('-inf') if ascending else float('inf'), reverse=not ascending)

    # Print the sorted data using tabulate
    print(tabulate(sorted_data, headers=headers, tablefmt="grid"))

    return sorted_data

Average Function

In [ ]:
def col_average(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
        print(f"Column '{column_name}' not found in the dataset.")

    column_index = headers.index(column_name)

    # Collect all non-null numeric entries for the column
    valid_entries = [float(row[column_index]) for row in data if row[column_index] is not None and is_number(row[column_index])]

    if not valid_entries:
        print(f"No valid numeric data found in column '{column_name}'.")

    # Calculate the average of these entries
    return sum(valid_entries) / len(valid_entries)

def is_number(value):
    """ Helper function to determine if a value is numeric. """
    try:
        float(value)
        return True
    except ValueError:
        return False


Count function

In [ ]:
def col_count(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
        print(f"Column '{column_name}' not found in the dataset.")

    column_index = headers.index(column_name)
    # Count non-null entries in the specified column
    return len([row[column_index] for row in data if row[column_index] is not None])


Min and Max Functions

In [ ]:
def col_min(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
        print(f"Column '{column_name}' not found in the dataset.")

    column_index = headers.index(column_name)

    # Retrieve all numeric entries and find the minimum
    try:
        return min(float(row[column_index]) for row in data if row[column_index] is not None and is_number(row[column_index]))
    except ValueError:
        print(f"No valid numeric data found in column '{column_name}' for minimum calculation.")

def col_max(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
       print(f"Column '{column_name}' not found in the dataset.")

    column_index = headers.index(column_name)

    # Retrieve all numeric entries and find the maximum
    try:
        return max(float(row[column_index]) for row in data if row[column_index] is not None and is_number(row[column_index]))
    except ValueError:
        print(f"No valid numeric data found in column '{column_name}' for maximum calculation.")


Sum Function

In [ ]:
def col_sum(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
        print(f"Column '{column_name}' not found in the dataset.")

    column_index = headers.index(column_name)

    # Sum all numeric entries in the specified column
    try:
        total = sum(float(row[column_index]) for row in data if row[column_index] is not None and is_number(row[column_index]))
        return total
    except ValueError:
        print(f"No valid numeric data found in column '{column_name}' for sum calculation.")



# Optimised Join

In [ ]:
'''def create_table(rows, pattern=1):
  # Pattern - 1 : [[1, 1], [2, 2], ... ]
  # Pattern - 2 : [[1, 1], [2, 1], ... ]
  return [[i, i if pattern == 1 else 1] for i in range(1, rows + 1)]

# Creating tables with headers and data
d1 = {'headers': ['col1', 'col2'], 'data': create_table(1000)}
d2 = {'headers': ['col1', 'col2'], 'data': create_table(1000, pattern=2)}
d3 = {'headers': ['col1', 'col2'], 'data': create_table(10000, pattern=1)}
d4 = {'headers': ['col1', 'col2'], 'data': create_table(10000, pattern=2)}

# Print sample data to verify
print("Data_1", d1)
print("Data_2", d2)
print("Data_3", d3)
print("Data_4", d4)

# Print length of sample data
print(len(d1['data']))
print(len(d2['data']))
print(len(d3['data']))
print(len(d4['data']))'''

'def create_table(rows, pattern=1):\n  # Pattern - 1 : [[1, 1], [2, 2], ... ]\n  # Pattern - 2 : [[1, 1], [2, 1], ... ]\n  return [[i, i if pattern == 1 else 1] for i in range(1, rows + 1)]\n\n# Creating tables with headers and data\nd1 = {\'headers\': [\'col1\', \'col2\'], \'data\': create_table(1000)}\nd2 = {\'headers\': [\'col1\', \'col2\'], \'data\': create_table(1000, pattern=2)}\nd3 = {\'headers\': [\'col1\', \'col2\'], \'data\': create_table(10000, pattern=1)}\nd4 = {\'headers\': [\'col1\', \'col2\'], \'data\': create_table(10000, pattern=2)}\n\n# Print sample data to verify\nprint("Data_1", d1)\nprint("Data_2", d2)\nprint("Data_3", d3)\nprint("Data_4", d4)\n\n# Print length of sample data\nprint(len(d1[\'data\']))\nprint(len(d2[\'data\']))\nprint(len(d3[\'data\']))\nprint(len(d4[\'data\']))'

In [ ]:
'''def basic_join(table1, table2, join_on1, join_on2):
    headers1 = table1['headers']
    headers2 = table2['headers']
    data1 = table1['data']
    data2 = table2['data']

    # Finding the index of the join columns
    join_index1 = headers1.index(join_on1)
    join_index2 = headers2.index(join_on2)

    # Performing the join
    joined_data = []
    for row1 in data1:
        for row2 in data2:
            if row1[join_index1] == row2[join_index2]:
                # Create a new row combining both rows, excluding the join column from the second table
                new_row = row1 + [item for idx, item in enumerate(row2) if idx != join_index2]
                joined_data.append(new_row)

    return joined_data

# eg usage
joined_data = basic_join(databases['Data_1'], databases['Data_3'], 'col1', 'col1')

# Display the joined data
for row in joined_data:
    print(row)'''

def optimised_join(table1, table2, join_on1, join_on2, numeric=False):
    # Determine if tables should be sorted or swapped
    if len(table1['data']) > len(table2['data']):
        table1, table2 = table2, table1
        join_on1, join_on2 = join_on2, join_on1

    # Get indices of the join columns from the headers
    join_index1 = table1['headers'].index(join_on1)
    join_index2 = table2['headers'].index(join_on2)

    # Create a lookup dictionary for the first table
    lookup = {}
    for row in table1['data']:
        key = int(row[join_index1]) if numeric else row[join_index1]
        if key not in lookup:
            lookup[key] = []
        lookup[key].append(row)

    # Perform the join using the lookup
    joined_data = []
    for row2 in table2['data']:
        key = int(row2[join_index2]) if numeric else row2[join_index2]
        if key in lookup:
            for row1 in lookup[key]:
                # Create a new row combining both rows, excluding the duplicate join key
                new_row = row1[:] + [item for idx, item in enumerate(row2) if idx != join_index2]
                joined_data.append(new_row)

    return joined_data, table1['headers'] + [col for col in table2['headers'] if col != join_on2]

# Example usage:
table1 = {'headers': ['id', 'name'], 'data': [['1', 'Alice'], ['2', 'Bob']]}
table2 = {'headers': ['id', 'age'], 'data': [['1', '24'], ['2', '30']]}
joined_data, headers = optimised_join(table1, table2, 'id', 'id', numeric=True)

"""
# Usage of the join function
joined_data, headers = optimised_join(d3, d4, 'col1', 'col1')
print(tabulate(joined_data, headers=headers, tablefmt='grid'))
"""

"\n# Usage of the join function\njoined_data, headers = optimised_join(d3, d4, 'col1', 'col1')\nprint(tabulate(joined_data, headers=headers, tablefmt='grid'))\n"

# Saving data back to csv files

In [ ]:
def save_databases_to_csv(databases, file_paths):
    for db_name, contents in databases.items():
        if db_name in file_paths:
            file_path = file_paths[db_name]
            with open(file_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(contents['headers'])  # write headers
                writer.writerows(contents['data'])    # write data rows
            print(f"Data for '{db_name}' saved to '{file_path}'.")
        else:
            print(f"No file path provided for '{db_name}'. Cannot save to CSV.")

save_databases_to_csv(databases, file_paths)

Data for 'data_1' saved to 'relation1.csv'.
Data for 'data_2' saved to 'relation2.csv'.
Data for 'data_3' saved to 'relation3.csv'.
Data for 'data_4' saved to 'relation4.csv'.
Data for 'visitors' saved to 'visitors.csv'.
Data for 'rides' saved to 'rides.csv'.


# Create Table Function and parsing code

In [ ]:
def create_table(table_name, columns):
  # Create the table if it doesn't exist
  databases[table_name] = {'headers': columns, 'data': []}
  print(f"Table '{table_name}' created successfully with columns {columns}")
  file_paths[table_name] = str(table_name)+'.csv'

"""
command = "create table Data_1 if not exists (ID, Name, Position);"
create_table_from_command(command, databases)

# Output the database to verify the table was created
print(databases) """

'\ncommand = "create table Data_1 if not exists (ID, Name, Position);"\ncreate_table_from_command(command, databases)\n\n# Output the database to verify the table was created\nprint(databases) '

# Create Parser

In [ ]:
# Create table query parse implementation
def create(command, pos):
  tables = [] # Stores table names
  values = [] # Stores values of the table
  d = False # Duplicate value
  cmd = str(command).split()
  if pos['create'] == 0 and pos['table'] == 1 and pos['if'] == 2 and pos['not'] == 3 and pos['exists'] == 4:
    tables = cmd[5]
    values = cmd[6:len(cmd)]
    values = clean(values)
    if 'primary' in values and 'key' in values:
      values.remove('primary')
      values.remove('key')
    if 'foreign' in values and 'key' in values:
      values.remove('foreign')
      values.remove('key')
    values = format(values)
    d = duplicates(values)
    if d == True:
      print('Duplicate values')
      return None
    if tables in db.keys(): # Checking if the table exists
      print('Table already exists')
      return None
    else: # Creating table
      # *************** Create table function implementation - use tables(list) and values(dict) as parameters
      db[tables] = values.keys() # DON'T REMOVE THIS
      create_table(tables, values.keys())
  else:
    print('Invalid Query')
  return None

# Insert Parser

In [ ]:
def insert_table(table_name, columns):
    # Check if the table exists in the databases dictionary
    if table_name in databases:
        # Check if the number of elements in columns matches the headers
        if len(columns) == len(databases[table_name]['headers']):
            # Append the new row to the table's data
            databases[table_name]['data'].append(columns)
        else:
            print("The number of provided columns does not match the table's headers.")
    else:
        print("Table does not exist in the database.")
    return None

# Insert table query parse implementation
def insert(command, pos):
  tables = [] # Stores table names
  values = [] # Stores values that are to be inserted
  cmd = str(command).split()
  if pos['insert'] == 0 and pos['into'] == 1 and pos['values']:
    tables = cmd[2]
    columns = cmd[3:pos['values']]
    columns = clean(columns)
    values = cmd[pos['values']+1:len(cmd)]
    values = clean(values)
    if tables not in db.keys(): # Checking if the table exists
      print('Table not found')
    elif len(values) != len(db[tables]):
      print('Missing fields')
    else: # Inserting data
      # *************** Insert table function implementation - use tables(list) and values(list) as parameters
      # print('Calling insert into table function')
      data = databases[tables]['data']
      headers = databases[tables]['headers']
      pk_col_name = keys[tables]['pk']
      pk_index = headers.index(pk_col_name)
      # Check for duplicate primary key
      if data != []:
          for row in data:
            if row[pk_index] == values[pk_index]:
              print('Duplicate key')
              return None
      insert_table(tables, values)
  else:
    print('Invalid Query')
  return None

# Simple Select

In [ ]:
# simple select
def simple_select(database, table_name, cols, limit=None):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    # Determine which columns to display
    if "*" in cols:
        selected_columns = headers  # All columns
        column_indices = list(range(len(headers)))
    else:
        selected_columns = [col for col in cols if col in headers]
        column_indices = [headers.index(col) for col in selected_columns]

    # Filter and reshape the data based on selected columns
    result_data = [[row[idx] for idx in column_indices] for row in data]

    # Apply the limit if there is one
    if limit is not None:
        result_data = result_data[:int(limit)]

    # Display the result in a formatted table using tabulate
    print(tabulate(result_data, headers=selected_columns, tablefmt="grid"))

    return result_data

# And or

In [ ]:
def select_with_and_or(database, table_name, cols, col1, op1, val1, col2, op2, val2, clause):
    # Check if the table exists
    if table_name not in database:
        print(f"Table '{table_name}' not found.")
        return

    # Retrieve table data
    table = database[table_name]
    headers = table['headers']
    data = table['data']

    # Check if columns exist in headers
    if not (col1 in headers and col2 in headers):
        missing_cols = [col for col in [col1, col2] if col not in headers]
        print(f"Column(s) {missing_cols} not found in the dataset.")
        return

    # Determine indices for the filter columns
    col1_index = headers.index(col1)
    col2_index = headers.index(col2)

    # Filter function based on operators
    def check_condition(value, operator, comparison_value):
        if operator == '=':
            return value == comparison_value
        elif operator == '>':
            return value > comparison_value
        elif operator == '<':
            return value < comparison_value
        elif operator == '>=':
            return value >= comparison_value
        elif operator == '<=':
            return value <= comparison_value
        elif operator == '!=':
            return value != comparison_value
        return False

    # Apply filters based on the clause
    if clause.lower() == 'and':
        filtered_data = [row for row in data if check_condition(row[col1_index], op1, val1) and check_condition(row[col2_index], op2, val2)]
    elif clause.lower() == 'or':
        filtered_data = [row for row in data if check_condition(row[col1_index], op1, val1) or check_condition(row[col2_index], op2, val2)]
    else:
        print("Invalid logical clause specified. Use 'AND' or 'OR'.")
        return

    # Select columns to display
    if '*' in cols:
        selected_columns = headers
        display_data = filtered_data
    else:
        if not set(cols).issubset(headers):
            print(f"Some specified columns {cols} are not found in the dataset.")
            return
        selected_columns = [col for col in cols if col in headers]
        display_data = [[row[headers.index(col)] for col in selected_columns] for row in filtered_data]

    # Display the results
    print(tabulate(display_data, headers=selected_columns, tablefmt="grid"))

# Order by

In [ ]:
def order_by(database, table_name, cols, column_name, ord="asc"):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    if column_name not in headers:
        print(f"Column '{column_name}' not found in the dataset.")

    # Determine which columns to display
    if "*" in cols:
        selected_columns = headers  # Select all columns if '*' is specified
        column_indices = range(len(headers))  # Use all indices
    else:
        selected_columns = [col for col in cols if col in headers]
        if not selected_columns:
            print("None of the specified columns are found in the dataset.")
        column_indices = [headers.index(col) for col in selected_columns]

    # Get the index of the column to sort by
    sort_index = headers.index(column_name)

    # Determine if the column data should be treated as numbers
    all_numbers = all(isinstance(row[sort_index], (int, float)) or (isinstance(row[sort_index], str) and row[sort_index].isdigit()) for row in data if row[sort_index] is not None)

    # Sort the data based on the specified column, converting to float if they're all numbers
    sorted_data = sorted(
        data,
        key=lambda x: (float('inf') if x[sort_index] is None else float(x[sort_index]) if all_numbers else x[sort_index]),
        reverse=ord != "asc"
    )

    # Filter and reshape the data based on selected columns after sorting
    sorted_filtered_data = [[row[idx] for idx in column_indices] for row in sorted_data]

    # Print the sorted and filtered data using tabulate
    print(tabulate(sorted_filtered_data, headers=selected_columns, tablefmt="grid"))

# Group by

In [ ]:
def select_group_by(database, table_name, cols, group_by_column):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    # Check if the group_by column is in the headers
    if group_by_column not in headers:
        print(f"Group by column '{group_by_column}' not found in the dataset.")

    group_by_index = headers.index(group_by_column)

    # Determine which columns to display
    if "*" in cols:
        selected_columns = headers  # Select all columns
        column_indices = range(len(headers))  # Use all indices
    else:
        selected_columns = [col for col in cols if col in headers]
        if not selected_columns:
            print("None of the specified columns are found in the dataset.")
        column_indices = [headers.index(col) for col in selected_columns]

    # Create a dictionary to hold grouped data
    grouped_data = defaultdict(list)

    # Populate the grouped data
    for row in data:
        group_key = row[group_by_index]
        selected_values = [row[idx] for idx in column_indices]
        grouped_data[group_key].append(selected_values)

    # Display the result
    for key, group in grouped_data.items():
        print(f"Group: {key}")
        print(tabulate(group, headers=selected_columns, tablefmt="grid"))

# Simple where clause

In [ ]:
def select_where(database, table_name, cols, col1, op1, val1):
    # Check if the table exists
    if table_name not in database:
        print(f"Table '{table_name}' not found.")
        return

    # Retrieve table data
    table = database[table_name]
    headers = table['headers']
    data = table['data']

    # Check if columns exist in headers
    if not (col1 in headers):
        missing_cols = [col for col in [col1] if col not in headers]
        print(f"Column(s) {missing_cols} not found in the dataset.")
        return

    # Determine indices for the filter columns
    col1_index = headers.index(col1)

    # Filter function based on operators
    def check_condition(value, operator, comparison_value):
        if operator == '=':
            return value == comparison_value
        elif operator == '>':
            return value > comparison_value
        elif operator == '<':
            return value < comparison_value
        elif operator == '>=':
            return value >= comparison_value
        elif operator == '<=':
            return value <= comparison_value
        elif operator == '!=':
            return value != comparison_value
        return False

    filtered_data = [row for row in data if check_condition(row[col1_index], op1, val1)]

    # Select columns to display
    if '*' in cols:
        selected_columns = headers
        display_data = filtered_data
    else:
        if not set(cols).issubset(headers):
            print(f"Some specified columns {cols} are not found in the dataset.")
            return
        selected_columns = [col for col in cols if col in headers]
        display_data = [[row[headers.index(col)] for col in selected_columns] for row in filtered_data]

    # Display the results
    print(tabulate(display_data, headers=selected_columns, tablefmt="grid"))

# Select Parser

In [ ]:
def columns(select_clause):
    select = select_clause.replace(',', ' ').split()
    cols = []
    aggr = None
    for part in select:
        if '(' in part and ')' in part:
            func_start = part.find('(')
            func_end = part.find(')')
            if func_start != -1 and func_end != -1 and func_start < func_end:
                func_name = part[:func_start].strip().lower()
                column_name = part[func_start+1:func_end].strip()
                aggr = (func_name, column_name)
        else:
            cols.append(part.strip())
    if aggr:
        return aggr[1], aggr[0]
    elif cols:
        return cols , None

def from_condition(from_clause):
    columns = str(from_clause).split(' ')
    table = columns[0]
    if 'limit' in from_clause:
      limit = columns[2]
      return table, limit
    else:
      return table, None

def where_condition(where_clause):
    where = str(where_clause).split(' ')
    clause = ''
    fnot = False
    wclause = False
    if 'not in' in where_clause:
      col = where[0]
      val = str(where[3:len(where)]).replace('(','').replace(')','').replace("'",'').replace('"','').replace(',','').replace('[','').replace(']','').split(' ')
      fnot = True
      wclause = True
      clause = 'not in'
    elif 'in' in where_clause and fnot == False:
      col = where[0]
      val = str(where[2:len(where)]).replace('(','').replace(')','').replace("'",'').replace('"','').replace(',','').replace('[','').replace(']','').split(' ')
      wclause = True
      clause = 'in'
    elif 'like' in where_clause:
      col = where[0]
      val = str(where[2:len(where)]).replace('(','').replace(')','').replace("'",'').replace('"','').replace(',','').replace('[','').replace(']','').split(' ')
      val = val[0]
      wclause = True
      clause = 'like'
    else:
      col = where[0]
      op = where[1]
      val = where[2]
    if wclause:
      return col, clause, val, True
    else:
      return col, op, val, False

def where_and_or(where_clause):
  where = str(where_clause).split(' ')
  clause = ''
  if 'and' in where_clause:
      col1 = where[0]
      op1 = where[1]
      val1 = where[2]
      col2 = where[4]
      op2 = where[5]
      val2 = where[6]
      wclause = True
      clause = 'and'
  elif 'and' or 'in' in where_clause:
      col1 = where[0]
      op1 = where[1]
      val1 = where[2]
      col2 = where[4]
      op2 = where[5]
      val2 = where[6]
      wclause = True
      clause = 'or'
  return col1, op1, val1, col2, op2, val2, clause

def order(order_by_clause):
    order = str(order_by_clause).split(' ')
    col = order[0]
    order_by = order[1]
    return col, order_by

def having(having_clause):
    having = str(having_clause).split(' ')
    col = having[0]
    op = having[1]
    val = having[2]
    return col, op, val

In [ ]:
class select_parser:
    def __init__(self, query):
        self.query = query.strip().rstrip(';')
        self.select_clause = None
        self.from_clause = None
        self.where_clause = None
        self.order_by_clause = None
        self.having_clause = None
        self.join_clause = None
        self.on_clause = None
        self.group_by_clause = None

    def parse(self):
        self.select_clause = self._parse_clause("SELECT", "FROM")
        self.from_clause = self._parse_clause("FROM", "WHERE|JOIN|GROUP BY|ORDER BY|HAVING|$")
        self.where_clause = self._parse_clause("WHERE", "GROUP BY|ORDER BY|HAVING|JOIN|$")
        self.group_by_clause = self._parse_clause("GROUP BY", "HAVING|ORDER BY|$")
        self.order_by_clause = self._parse_clause("ORDER BY", "HAVING|$")
        self.having_clause = self._parse_clause("HAVING", "$")
        self.join_clause = self._parse_clause("JOIN", "ON")
        self.on_clause = self._parse_clause("ON", "WHERE|GROUP BY|ORDER BY|HAVING|$")

    def _parse_clause(self, start_keyword, end_keyword_pattern):
        pattern = rf"{start_keyword}\s+(.*?)(?=(?:{end_keyword_pattern})|\Z)"
        if isinstance(self.query, str):
            match = re.search(pattern, self.query, re.IGNORECASE)
            if match:
                return match.group(1).strip()
        return None

    def display_parsed_components(self):
        and_or = False
        cols, aggr = columns(self.select_clause)
        # cols -> columns of the table
        # aggr -> aggregations if exists else None
        #print("Columns:", cols)
        #print("Aggregate Function:", aggr)
        table, limit = from_condition(self.from_clause)
        # table -> table name
        # limit -> limit if exists else None
        #print("Table:", table)
        #print("Limit:", limit)
        if self.where_clause:
          if 'and' in self.where_clause or 'or' in self.where_clause:
            col1, op1, val1, col2, op2, val2, clause = where_and_or(self.where_clause)
            #print("Column 1:", col1)
            #print("Operation 1:", op1)
            #print("Value 1:", val1)
            #print("Column 2:", col2)
            #print("Operation 2:", op2)
            #print("Value 2:", val2)
            #print("Clause:", clause)
            and_or = True
          else:
            col, cond, val, flag = where_condition(self.where_clause)
            #print("Column:", col)
            #print("Condition:", cond)
            #print("Value:", val)
        if self.order_by_clause:
          col, ord = order(self.order_by_clause)
          #print("Column:", col)
          #print("Order:", ord)
        if self.having_clause:
          col, op, val = having(self.having_clause)
          #print("Column:", col)
          #print("Operation:", op)
          #print("Value:", val)
        #print("GROUP BY Clause:", self.group_by_clause)
        group_by = self.group_by_clause
        #print("JOIN Clause:", self.join_clause)
        join = self.join_clause
        #print("ON Clause:", self.on_clause)
        on = self.on_clause
        if self.select_clause and self.from_clause and aggr != None:
        # *************** Create a function to implement select with from - use cols, aggr, table, limit (Note: If there is no limit in the query this will be none, else will be a number)
        # Should support the following queries :
        # SELECT COUNT(*) FROM employees;
        # SELECT SUM(id) FROM employees;
        # SELECT AVG(id) FROM employees;
        # SELECT MAX(id) FROM users;
        # SELECT MIN(id) FROM employees;
          #print('Calling select aggregation clause function')
          if aggr == "count":
              res = col_count(databases, table, cols)
          elif aggr == "avg":
              res = col_average(databases, table, cols)
          elif aggr == "max":
              res = col_max(databases, table, cols)
          elif aggr == "min":
              res = col_min(databases, table, cols)
          elif aggr == "sum":
              res = col_sum(databases, table, cols)
          h = str(aggr)+'('+str(cols)+')'
          print(tabulate([[res]], headers=[h], tablefmt="grid"))

        elif self.select_clause and self.from_clause and not self.join_clause and not self.where_clause and not self.order_by_clause and not self.group_by_clause:
          # *************** Create a function to implement select with from - use cols, table, limit
          # Note: If there is no limit in the query this will be none, else will be a number
          # Should support the following queries :
          # SELECT * FROM employees;
          # SELECT first_name, last_name FROM employees;
          # SELECT * FROM employees LIMIT 10;
          #print('Calling simple select clause function')
          simple_select(databases, table, cols, limit)

        if self.select_clause and self.from_clause and self.join_clause:
          #print('Calling select join clause function')
          on = str(on).split('=')
          cond1 = str(on[0]).replace(' ', '').split('.')
          cond1 = cond1[1]
          cond2 = str(on[1]).replace(' ', '').split('.')
          cond2 = cond2[1]
          joined_data, headers = optimised_join(databases[table], databases[join], cond1, cond2)
          print(tabulate(joined_data, headers=headers, tablefmt='grid'))

        if self.select_clause and self.from_clause and self.where_clause and and_or == True:
          # *************** Create a function to implement select with 'and' or 'or' clause - use cols, table, col1, op1, val1, col2, op2, val2, clause as parameters
          # Clause -> decides 'and' or 'or' clause
          # Should support the following queries :
          # SELECT * FROM users WHERE name = 'John' AND age = 25;
          # SELECT * FROM users WHERE name = 'John' OR age = 25;
          #print('Calling and or clause function')
          select_with_and_or(databases, table, cols, col1, op1, val1, col2, op2, val2, clause)

        elif self.select_clause and self.from_clause and self.where_clause:
          # *************** Create a function too implement select with 'in', 'not in', 'like' or simple 'where' clause - use cols, table, col, cond, val and flag as parameters
          # if flag == True -> cond will be 'in', 'not in' or 'like' clause
          # if flag == False -> cond will be operation (>,<,=)
          # Should support the following queries :
          # SELECT * FROM employees WHERE name = 'Sales';
          # SELECT * FROM users WHERE name in ('John' 'Jane');
          # SELECT * FROM users WHERE name not in ('John' 'Jane');
          # SELECT * FROM users WHERE name LIKE '%ab%';
          #print('Calling in, not in, like clause function')
          select_where(databases, table, cols, col, cond, val)

        if self.select_clause and self.from_clause and self.order_by_clause:
          # *************** Create a function too implement select with 'order' clause - use cols, table, col, ord as parameters
          # Should support the following queries :
          # SELECT * FROM employees ORDER BY name DESC;
          # SELECT * FROM employees ORDER BY name ASC;
          #print('Calling select order by clause function')
          order_by(databases, table, cols, col, ord)

        elif self.select_clause and self.from_clause and self.group_by_clause:
          # *************** Create a function too implement select with 'group' clause - use cols, table, group_by as parameters
          # Should support the following queries :
          # SELECT id FROM employees GROUP BY name;
          #print('Calling select group by clause function')
          select_group_by(databases, table, cols, group_by)
        return

# Select table query parse implementation
def select(command, pos):
  cmd = str(command)
  parser = select_parser(cmd)
  parser.parse()
  parser.display_parsed_components()
  return None

# Alter functions

In [ ]:
def print_column_names(database):
    for table_name, table_info in database.items():
        headers = table_info['headers']
        print(f"Table '{table_name}' columns: {headers}")

def add_column(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    # Access the table
    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    # Check if the column already exists
    if column_name in headers:
        print(f"Column '{column_name}' already exists in the table.")

    # Add the new column to headers
    headers.append(column_name)
    print(headers)

    # Append a None value to each row in the data
    for row in data:
        row.append(None)

    print(f"Column '{column_name}' added successfully.")
    print_column_names(database)


def drop_column(database, table_name, column_name):
    if table_name not in database:
        print(f"Table '{table_name}' not found.")

    # Access the table
    table_dict = database[table_name]
    headers = table_dict['headers']
    data = table_dict['data']

    # Check if the column exists
    if column_name not in headers:
        print(f"Column '{column_name}' does not exist in the table.")

    # Find the index of the column to be removed
    column_index = headers.index(column_name)

    # Remove the column from headers
    headers.pop(column_index)

    # Remove the corresponding data from each row
    for row in data:
        row.pop(column_index)

    print(f"Column '{column_name}' dropped successfully.")
    print_column_names(database)


# Alter Parser

In [ ]:
# Alter table query parse implementation
def alter(command, pos):
  tables = [] # Stores table names
  values = [] # Stores values that have to be altered
  op = '' # Stores operatio to be performed in alter query
  cmd = str(command).split()
  if pos['alter'] == 0 and pos['table'] == 1 and 'add' in pos:
    op = 'alter add'
    tables = cmd[2]
    values = cmd[4:len(cmd)]
    values = clean(values)
    values = format(values)
    if tables not in db.keys(): # Checking if the table exists
      print('Table not found')
    else: # Altering data - add
      # *************** Alter add table function implementation - use op(string), tables(list) and values(dict) as parameters
      #print('Calling alter add function')
      values = list(values.keys())
      add_column(databases, tables, values[0])
  elif pos['alter'] == 0 and pos['table'] == 1 and 'drop' in pos and 'column' in pos:
    op = 'alter drop'
    tables = cmd[2]
    values = cmd[5:len(cmd)]
    values = clean(values)
    print(values)
    if tables not in db.keys(): # Checking if the table exists
      print('Table not found')
    elif not values not in db[tables]: # Checking if the columns exists
      print('Column not found')
    else: # Altering data - drop
      # *************** Alter drop table function implementation - use op(string), tables(list) and values(dict) as parameters
      #print('Calling alter drop function')
      drop_column(databases, tables, values[0])
  else:
    print('Invalid Query')
    # drop_table(databases, tables)
  return None


# Show Tables - Not Select *

In [ ]:
# Show tables query implementation
def show_all_tables(db):
  table_names = [[name] for name in db.keys()]
  print(tabulate(table_names, headers=["Tables"], tablefmt="grid"))
  return None

# Main code for parser

In [ ]:
# Main function to call all operations
def main(cmd, dynamic_db):
  cmd = cmd.strip().lower() # Input command in lowercase
  command = sqlparse.parse(cmd) # Parsing input query
  pos = positions(command) # Getting positions of tokens in parsed query
  command = str(command[0])
  if "create table" in cmd:
    create(command, pos) # Create table
  elif "insert into" in cmd:
    insert(command, pos) # Insert into table
  elif "select" in cmd:
    select(command, pos) # Select from table
  elif "update" in cmd:
    update(command, pos) # Update table
  elif "alter" in cmd:
    alter(command, pos) # Alter table
  elif "delete" in cmd:
    delete(command, pos) # Delete table
  elif "drop" in cmd:
    drop(command, pos) # Drop table
  elif "show tables" in cmd:
    show_all_tables(db) # Show tables
  else:
    print("Invalid command") # Invalid input
  return None


# Command Line Interface

In [ ]:
# Calling main function
if __name__ == "__main__":
    databases, file_paths = given_sample_upload()
    # output will be first 10 rows of each database
    dynamic_db = DynamicDB(databases)  # Assuming 'databases' is your data loaded from CSVs or similar.
    dynamic_db.load_indices()  # This method should initialize the indices.


    while True:
        cmd = input("sql> ") # Getting input
        st = time.time() # Start of execution time of query
        if cmd.lower() == "exit;": # Exit command
            save_databases_to_csv(databases, file_paths) # Change with load data function ********
            break
        main(cmd, dynamic_db) # Calling main to perform operations
        et = time.time() # End of execution time of query
        dur = et - st # Execution time of query
        print(f"({dur:.2f}) sec")
        print()

sql> update data_1 set col1 = 10 where col1 = 3;
Updated 1 row(s) where col1 = 3 setting col1 to 10.
(0.00) sec

sql> select * from data_1 limit 4;
+--------+--------+
|   col1 |   col2 |
+========+========+
|      1 |      1 |
+--------+--------+
|      2 |      2 |
+--------+--------+
|     10 |      3 |
+--------+--------+
|      4 |      4 |
+--------+--------+
(0.00) sec

sql> update data_1 set col1 = 100 where col1 = 10;
Updated 1 row(s) where col1 = 10 setting col1 to 100.
(0.00) sec

sql> select * from data_1 limit 4;
+--------+--------+
|   col1 |   col2 |
+========+========+
|      1 |      1 |
+--------+--------+
|      2 |      2 |
+--------+--------+
|     10 |      3 |
+--------+--------+
|      4 |      4 |
+--------+--------+
(0.01) sec

sql> update data_1 set col1 = 1000 where col1 = 10;
Updated 1 row(s) where col1 = 10 setting col1 to 1000.
(0.00) sec

sql> select * from data_1 limit 4;
+--------+--------+
|   col1 |   col2 |
+========+========+
|      1 |      1 |
+-

KeyboardInterrupt: Interrupted by user

update data_1 set col1 = 10 where col1 = 3;

# Queries

Queries:
show tables;

1. Create:
create table if not exists parks (pid integer primary key, location varchar(10));

2. Selects:

select * from visitors;

select * from rides limit 4;

select max(age) from visitors;

select min(age) from visitors;

select sum(times_ridden) from rides;

select max(times_ridden) from rides;

select min(times_ridden) from rides;

select avg(times_ridden) from rides;

select count(ride_name) from rides;

select * from visitors join rides on visitors.visitor_id = rides.visitor_id;

select * from data_1 join data_2 on data_1.col1 = data_2.col1;

select * from data_3 join data_4 on data_3.col1 = data_4.col1;

3. Inserts:
insert into visitors values (6, 'Sophia', 6, 'Elsa');

insert into rides values (317, 6, 'ElsaPalaceTour', 3);


4. Selects after inserts:
select ride_name from rides where ride_id = 305;

select age from visitors order by age asc;

select visitor_id from visitors order by visitor_id asc;

select age from visitors order by age desc;

select visitor_id from visitors order by visitor_id desc;

select ride_name, visitor_id from rides group by visitor_id;

select ride_name, times_ridden from rides where times_ridden >
2 or ride_name = 'AmazingSpiderRide';

select ride_name, times_ridden from rides where times_ridden = 1 or ride_name = 'AmazingSpiderRide';

5. Updates: -
update visitors set age = 10 where visitor_id = 3;

update visitors set age = 8 where visitor_id = 5;

6. Alters:
alter table rides add park_name varchar(20);

alter table rides add park_status varchar(20);

alter table rides drop column park_name;

alter table rides drop column park_status;

7. Delete:
delete from rides where ride_id = 309;

8. Drop:
drop table parks;


show tables;

CREATE TABLE IF NOT EXISTS users (id INT PRIMARY KEY, name TEXT, age INT, dept varchar(40));

DROP TABLE data_4;

SELECT * from data_1 LIMIT 10;

SELECT col1 from data_1 order by col1 DESC;

SELECT col1, col2 from data_1 by col1 DESC;

ALTER TABLE data_1 ADD col3 VARCHAR(255);

SELECT AVG(col1) FROM data_1;

ALTER TABLE data_1 DROP COLUMN col3; <- GIVES ERRORS